In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp rics

# RIC's

In [ ]:
#export
import numpy as np
import qutip as qt
from scipy.stats import ortho_group

from qbism.povm import *
from qbism.random import *
from qbism.kraus import *

One can also consider POVM's on real valued Hilbert spaces. RIC-POVM's (real informationally complete POVM's) will have $\frac{d(d+1)}{2}$ elements (unlike the complex case, where they would be $d^2$ elements).

SIC-POVM's in real Hilbert spaces correspond to sets of *real* equiangular lines, and unlike in the complex case, they can be proved *not* to exist in certain dimensions.

For purposes of testing out RIC's, let's define some useful functions:

In [ ]:
#export
def real_rand_ket(d):
    r"""
    Generates a random ket in real Hilbert space of dimension $d$.
    """
    return qt.Qobj(np.random.randn(d)).unit()

In [ ]:
#export
def real_rand_dm(d):
    r"""
    Generates a random density matrix for a real Hilbert space of dimension $d$.
    """
    return qt.Qobj(qt.rand_dm(d).full().real)

In [ ]:
#export
def rand_symmetric(d):
    r"""
    Generates a random $d \times d$ symmetric matrix. These matrices correspond to observables in real quantum mechanics, being the real analogue of Hermitian matrices: $\hat{S} = \hat{S}^{T}$.
    """
    M = qt.Qobj(np.random.randn(d,d))
    return M*M.trans() + M.trans()*M

In [ ]:
#export
def rand_orthogonal(d):
    r"""
    Generates a random $d \times d$ orthogonal matrix. These matrices correspond to time evolution in real quantum mechanics, being the real analogue of unitary matrices: $\hat{S}\hat{S}^{T} = \hat{I}$.
    """
    return qt.Qobj(ortho_group.rvs(d))

Let's generate a random RIC and check that it behaves like the more usual complex IC-POVM's we're used to. First, let's check that we can go back and forth between density matrices and probabilities:

In [ ]:
d = 3
povm = random_haar_povm(d, real=True)
phi = povm_phi(povm)
rho = real_rand_dm(d)
p = dm_probs(rho, povm)
assert np.allclose(rho, probs_dm(p, povm))

Then let's compare classical and quantum probabilities for some observable represented by a symmetric matrix:

In [ ]:
S = rand_symmetric(d)
vn = [v*v.dag() for v in S.eigenstates()[1]]

R = conditional_probs(vn, povm)
classical_probs = R @ p
quantum_probs = R @ phi @ p

post_povm_rho = sum([(e*rho).tr()*(e/e.tr()) for e in povm])
assert np.allclose(classical_probs, [(v*post_povm_rho).tr() for v in vn])
assert np.allclose(quantum_probs, [(v*rho).tr() for v in vn])

And finally, let's check out time evolution under an othogonal matrix:

In [ ]:
O = rand_orthogonal(d)
assert np.allclose(dm_probs(O*rho*O.trans(), povm), povm_map([O], povm) @ phi @ p)